# Imports

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

Matplotlib created a temporary config/cache directory at /var/folders/kv/34jyv69d3lq7qnz8d_2l_s4w0000gq/T/matplotlib-oy2blxoq because the default path (/Users/ibemmene/.matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
Fontconfig warning: ignoring UTF-8: not a valid region tag


In [2]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score

from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold

# Functions & Parameters

In [3]:
SEED = 23
data_dir = Path("../Daten/data_to_use")
paper_data_dir = Path("../../Data Garcia Paper")
assert data_dir.exists()
assert paper_data_dir.exists()

# Investigation on the Data of the Paper

**TODO: Make it formal and chek for all datasets**

Make sure that we know which data we are working with. And that the statements in the paper are correctly understood. Then the data is save to use.

## Check if the Datasets are identical

In [4]:
df_japan = pd.read_json(data_dir/"japan_train.json")
df_japan_paper = pd.read_csv(paper_data_dir/"DB_Collusion_Japan_processed.csv")

In [5]:
print(len(df_japan))
print(len(df_japan_paper))

10185
13515


In [6]:
print(len(set(df_japan.project.tolist())))
print(len(set(df_japan_paper.Tender.tolist())))

810
1080


In [7]:
df_japan_paper[~(df_japan_paper.Tender.isin(list(set(df_japan.project.tolist()))))]

Tender        Date  Site  Pre-Tender Estimate (PTE)  Competitors  \
40          6  1052870400    10                  8820000.0          955   
41          6  1052870400    10                  8820000.0          169   
42          6  1052870400    10                  8820000.0         1115   
43          6  1052870400    10                  8820000.0         1579   
44          6  1052870400    10                  8820000.0          131   
...       ...         ...   ...                        ...          ...   
13447    1448  1174003200   159                121768500.0         1540   
13448    1448  1174003200   159                121768500.0          533   
13449    1448  1174003200   159                121768500.0          179   
13450    1448  1174003200   159                121768500.0          999   
13451    1448  1174003200   159                121768500.0         1031   

         Name competitors  Winner  Collusive_competitor  Bid_value  \
40        SHIMMARUKODOKEN       1                     0    8240000   
41        KOTATSURUKIKAKU       0                     0    8270000   
42             OKURAKOGYO       0                     0    8290000   
43            SHINJOKOGYO       0                     0    8300000   
44        MIYAHIROSHIGUMI       0                     0    8310000   
...                   ...     ...                   ...        ...   
13447          SANIDOBOKU       0                     0   97530000   
13448      TAKARAKENSETSU       0                     0   97666600   
13449    KOTAKARAKENSETSU       0                     0   98159200   
13450    YOITSUKIKENSETSU       0                     0   98500000   
13451  ISHIKAWADOKENKOGYO       0                     0  106720000   

       Difference Bid/PTE  Number_bids Period investigation by JFTC      CV  \
40               0.980952           10               pre-inspection  0.0045   
41               0.984524           10               pre-inspection  0.0045   
42               0.986905           10               pre-inspection  0.0045   
43               0.988095           10               pre-inspection  0.0045   
44               0.989286           10               pre-inspection  0.0045   
...                   ...          ...                          ...     ...   
13447            0.840993           18               post-amendment  0.0283   
13448            0.842171           18               post-amendment  0.0283   
13449            0.846419           18               post-amendment  0.0283   
13450            0.849358           18               post-amendment  0.0283   
13451            0.920238           18               post-amendment  0.0283   

          SPD   DIFFP      RD    KURT    SKEW  KSTEST  \
40     0.0146  0.0036  1.0258  0.0641 -0.3211  0.2167   
41     0.0146  0.0036  1.0258  0.0641 -0.3211  0.2167   
42     0.0146  0.0036  1.0258  0.0641 -0.3211  0.2167   
43     0.0146  0.0036  1.0258  0.0641 -0.3211  0.2167   
44     0.0146  0.0036  1.0258  0.0641 -0.3211  0.2167   
...       ...     ...     ...     ...     ...     ...   
13447  0.1529  0.0300  0.0000  9.9579  2.5918  0.5253   
13448  0.1529  0.0300  0.0000  9.9579  2.5918  0.5253   
13449  0.1529  0.0300  0.0000  9.9579  2.5918  0.5253   
13450  0.1529  0.0300  0.0000  9.9579  2.5918  0.5253   
13451  0.1529  0.0300  0.0000  9.9579  2.5918  0.5253   

       Collusive_competitor_original  
40                                 0  
41                                 0  
42                                 0  
43                                 0  
44                                 0  
...                              ...  
13447                              0  
13448                              0  
13449                              0  
13450                              0  
13451                              0  

[3330 rows x 20 columns]

In [8]:
df_japan_paper[(df_japan_paper.Tender.isin(list(set(df_japan.project.tolist()))))].Bid_value

0        11169000
1        11427370
2        11456000
3        11487700
4        11492000
           ...   
13510     3660000
13511     3663000
13512     3670000
13513     3670000
13514     3927000
Name: Bid_value, Length: 10185, dtype: int64

In [9]:
missing_projects = df_japan_paper[~(df_japan_paper.Tender.isin(list(set(df_japan.project.tolist()))))].Tender.tolist()

In [10]:
(df_japan.bid-df_japan_paper[(df_japan_paper.Tender.isin(list(set(df_japan.project.tolist()))))].Bid_value).sum()

0

Remaining bids are identical. 
**TODO**: Question to Hannes why has the rest been cut out? Oh wait this is probably my own test data i put away :D Check it: Get project numbers of TUD

In [11]:
TUD_dir = Path("../Daten/TUD")
assert TUD_dir.exists()

In [12]:
df_japan = pd.read_json(TUD_dir/"japan_unseen.json")

In [13]:
TUD_sorted = list(set(df_japan.project.tolist()))
missing_sorted = list(set(missing_projects))
TUD_sorted.sort()
missing_sorted.sort()

In [14]:
print(len(missing_sorted))
print(len(TUD_sorted))

270
270


In [15]:
np.array_equal(np.asarray(missing_sorted), np.asarray(TUD_sorted))

True

**The two Datasets are identical**

## Check 11% rule collusive competitor original

And mainly if this part in the paper has been properly understood.

In [16]:
df_japan_paper_grouped = df_japan_paper.groupby(by="Tender").sum().reset_index()

In [17]:
percentages = np.asarray(df_japan_paper_grouped[df_japan_paper_grouped.Collusive_competitor != 0].Collusive_competitor_original.tolist())/np.asarray(df_japan_paper_grouped[df_japan_paper_grouped.Collusive_competitor != 0].Collusive_competitor.tolist())

In [18]:
print(len(percentages))
print(len(percentages[percentages >0.11]))

123
123


**All Tenders which have been labelled as collusive, have more than 11% collusive competitors**

**Comparison of statistical features in the two datasets**

Identical: CV, SPD, DIFFP, SKEW

Same name but different values: RD and KS/KSTEST

--> Needs to be recalculated or check with Hannes why they're different

# Data Wrangling such that it is ready to be analyzed

Apply the same train/test split as with our Japan data

In [19]:
df_japan = pd.read_json(data_dir/"japan_train.json")
df_japan_paper = pd.read_csv(paper_data_dir/"DB_Collusion_Japan_processed.csv")

df_japan_paper_test = df_japan_paper[df_japan_paper.Tender.isin(TUD_sorted)]
df_japan_paper = df_japan_paper[~df_japan_paper.Tender.isin(TUD_sorted)]

print(len(df_japan_paper))
print(len(df_japan_paper_test))

10185
3330


Prepare the data with the needed features.
- Tender
- Bid_value
- Number_bids
- CV
- SPD
- DIFFP
- RD
- KURT
- SKEW
- KSTEST
Prediction variable: 
- Collusive_competitor_original

**Normalize Bids per auction**

In [20]:
df_japan_paper["Bid_value"] = df_japan_paper.groupby("Tender").Bid_value.transform(lambda x: (x-x.min())/x.std())

# Data analysis with the full data (each row)

Let's just check what happens if we throw the data into RandomForest (does not need OneHotEncoding)

In [21]:
conf_ML = [
    ('Dummy', DummyClassifier(), {'Dummy__strategy' : ["stratified"], 'Dummy__random_state': [SEED]}),
    ('Tree', DecisionTreeClassifier(), {'Tree__max_depth': range(5,20), 'Tree__random_state': [SEED]}),
    ('Forest', RandomForestClassifier(), {'Forest__max_depth': range(5,20), 'Forest__random_state': [SEED]}),
    #('GradientBoost', GradientBoostingClassifier(), {'GradientBoost__max_depth': range(1,10), 'GradientBoost__random_state': [SEED]}),
    #('MLP', MLPClassifier(), {'MLP__hidden_layer_sizes': range(1,4), "MLP__random_state": [SEED]}),
    #('Logistic', LogisticRegression(), {'Logistic__random_state': [SEED]}),
    #('Poly_SVM', SVC(), {'Poly_SVM__C' : [1.0, 2.5, 5.0, 7.5, 10.0], 'Poly_SVM__kernel' : ['poly'], 'Poly_SVM__degree' : range(1,3), 'Poly_SVM__random_state': [SEED]}),
    #('SVM', SVC(), {'SVM__C' : np.logspace(2,3, 10), 'SVM__gamma': np.logspace(-1,1, 10), 'SVM__kernel' : ['rbf'], 'SVM__random_state': [SEED]})
]

In [22]:
df_japan_paper_grouped = df_japan_paper.groupby("Tender").head(1)

In [23]:
df_japan_paper_grouped.head()

Tender        Date  Site  Pre-Tender Estimate (PTE)  Competitors  \
0        2  1050537600   115                 14952000.0         1305   
10       3  1051228800   115                 23667000.0          125   
20       4  1052352000    84                 38800000.0         1293   
30       5  1052697600    56                 15800000.0          852   
50       7  1052870400   270                  8080000.0          794   

      Name competitors  Winner  Collusive_competitor  Bid_value  \
0       HEISEIKAIHATSU       0                     0        0.0   
10        GIMAKENSETSU       1                     0        0.0   
20       HAMASHINDOKEN       1                     0        0.0   
30  SANKOKENSETSUKOGYO       1                     0        0.0   
50    MINAMOTOKENSETSU       1                     0        0.0   

    Difference Bid/PTE  Number_bids Period investigation by JFTC      CV  \
0             0.784340           10               pre-inspection  0.1021   
10            0.801540           10               pre-inspection  0.0916   
20            0.985052           10               pre-inspection  0.0040   
30            0.970253           10               pre-inspection  0.0082   
50            0.968131           10               pre-inspection  0.0103   

       SPD   DIFFP      RD    KURT    SKEW  KSTEST  \
0   0.2982  0.0231  0.2375 -1.2043  0.8118  0.4030   
10  0.2515  0.0025  0.0245 -1.6380 -0.4335  0.2896   
20  0.0137  0.0065  1.6591  1.7464 -1.0795  0.3740   
30  0.0282  0.0027  0.4312  0.4611  0.6144  0.2453   
50  0.0314  0.0067  0.6470 -0.1258 -0.9291  0.3265   

    Collusive_competitor_original  
0                               0  
10                              0  
20                              0  
30                              0  
50                              0

In [24]:
X_grouped = df_japan_paper_grouped.drop(columns = ["Collusive_competitor"])
y_grouped = df_japan_paper_grouped["Collusive_competitor"]

In [25]:
features = ["Tender", "Winner", "Bid_value", "Difference Bid/PTE", "Number_bids", "CV", "SPD", "DIFFP", "RD", "KURT", "SKEW", "KSTEST"]

In [26]:
X = df_japan_paper[features]
y = df_japan_paper["Collusive_competitor_original"]

Make a Stratified K-Fold Split (grouping is necessary to make sure that everything of the same Tender stays together)

In [27]:
X.tail()

Tender  Winner  Bid_value  Difference Bid/PTE  Number_bids      CV  \
13510    1454       0   0.532004            0.979133           14  0.0205   
13511    1454       0   0.571904            0.979936           14  0.0205   
13512    1454       0   0.665005            0.981808           14  0.0205   
13513    1454       0   0.665005            0.981808           14  0.0205   
13514    1454       0   4.083132            1.050562           14  0.0205   

          SPD   DIFFP   RD     KURT    SKEW  KSTEST  
13510  0.0848  0.0083  0.0  12.7244  3.4937  0.7657  
13511  0.0848  0.0083  0.0  12.7244  3.4937  0.7657  
13512  0.0848  0.0083  0.0  12.7244  3.4937  0.7657  
13513  0.0848  0.0083  0.0  12.7244  3.4937  0.7657  
13514  0.0848  0.0083  0.0  12.7244  3.4937  0.7657

In [28]:
X.loc[13514]

Tender                1454.000000
Winner                   0.000000
Bid_value                4.083132
Difference Bid/PTE       1.050562
Number_bids             14.000000
CV                       0.020500
SPD                      0.084800
DIFFP                    0.008300
RD                       0.000000
KURT                    12.724400
SKEW                     3.493700
KSTEST                   0.765700
Name: 13514, dtype: float64

In [29]:
np.arange(len(X))

array([    0,     1,     2, ..., 10182, 10183, 10184])

In [34]:
skf = StratifiedKFold(n_splits=5,random_state = SEED, shuffle=True)
train_tenders = []
test_tenders = []


train_indices = []
test_indices = []
for train_index, test_index in skf.split(X_grouped, y_grouped):
    train_tenders.append(X_grouped.Tender.iloc[train_index].tolist())
    test_tenders.append(X_grouped.Tender.iloc[test_index].tolist())
    
    train_indices.append(np.arange(len(X))[X.Tender.isin(X_grouped.Tender.iloc[train_index].tolist())])
    test_indices.append(np.arange(len(X))[X.Tender.isin(X_grouped.Tender.iloc[test_index].tolist())])                     

    
    

In [35]:
my_cv = []
for i in range(5):
    my_cv.append((train_indices[i], test_indices[i]))

In [36]:
my_cv[0]

(array([   10,    11,    12, ..., 10182, 10183, 10184]),
 array([    0,     1,     2, ..., 10168, 10169, 10170]))

In [37]:
y.iloc[my_cv[0][1]].sum()

147

In [38]:
y.iloc[my_cv[1][1]].sum()

177

In [39]:
y.iloc[my_cv[4][1]].sum()

157

In [40]:
len(y.iloc[my_cv[0][1]])

2084

In [41]:
len(y.iloc[my_cv[1][1]])

2037

In [42]:
def evaluate_models(X,y, configuration, SEED = SEED):
    model_scores = []
    models = []
    results = []
    full_results = []
    for name, algo, params in configuration:
        print(f'Trying {name} ...')
        # Extend model with normalisation step (here we use z-normalisation)
        #("scaler", StandardScaler()) # only needed in ML hyper parameter tuning
        pipeline = Pipeline([("scaler", StandardScaler()),(name, algo)])
        # Instantiate grid search object and pass params, number of folds and scoring-function
        classifier = GridSearchCV(pipeline, param_grid=params, cv=my_cv, scoring=['f1', 'precision', 'recall'], refit = 'f1', return_train_score = True)
        # Fit cross validation to our training data: within this function, all configurations are tried
        classifier.fit(X, y)
        print(f'  - Best hyperparameter: {classifier.best_params_}')
        print(f'  - Best F1 score: {classifier.best_score_:.3f}')
        print(f'  - F1 Result for each parametrization in train:', [f'{x:.2f}' for x in classifier.cv_results_['mean_train_f1']])
        print(f'  - F1 Result for each parametrization in test:', [f'{x:.2f}' for x in classifier.cv_results_['mean_test_f1']])
        print(f'  - precision Result for each parametrization in train:', [f'{x:.2f}' for x in classifier.cv_results_['mean_train_precision']])
        print(f'  - precision for each parametrization in test:', [f'{x:.2f}' for x in classifier.cv_results_['mean_test_precision']])
        print(f'  - recall Result for each parametrization in train:', [f'{x:.2f}' for x in classifier.cv_results_['mean_train_recall']])
        print(f'  - recall Result for each parametrization in test:', [f'{x:.2f}' for x in classifier.cv_results_['mean_test_recall']])
        model_scores.append(classifier.best_score_)
        models.append(classifier.best_estimator_)
    
        index = classifier.best_index_
        df_results = pd.DataFrame({
            "algorithm": name,
            "train_f1": [classifier.cv_results_['mean_train_f1'][index]], "test_f1": [classifier.cv_results_['mean_test_f1'][index]],
            "train_precision": [classifier.cv_results_['mean_train_precision'][index]], "test_precision": [classifier.cv_results_['mean_test_precision'][index]],
            "train_recall": [classifier.cv_results_['mean_train_recall'][index]], "test_recall": [classifier.cv_results_['mean_test_recall'][index]],
            "params": [classifier.best_params_]
            })
        results.append(df_results)
    
        df_full_results = pd.DataFrame({
            "algorithm": name,
            "train_f1": classifier.cv_results_['mean_train_f1'], "test_f1": classifier.cv_results_['mean_test_f1'],
            "train_precision": classifier.cv_results_['mean_train_precision'], "test_precision": classifier.cv_results_['mean_test_precision'],
            "train_recall": classifier.cv_results_['mean_train_recall'], "test_recall": classifier.cv_results_['mean_test_recall'],
            "params": classifier.cv_results_["params"]
            })
        full_results.append(df_full_results)
    
    results = pd.concat(results)
    results = results.reset_index(drop = True)

    full_results = pd.concat(full_results)
    full_results = full_results.reset_index(drop = True)
    
    return results, full_results

In [43]:
results_japan, full_results_japan = evaluate_models(X,y, conf_ML, SEED = 45)

Trying Dummy ...
  - Best hyperparameter: {'Dummy__random_state': 23, 'Dummy__strategy': 'stratified'}
  - Best F1 score: 0.075
  - F1 Result for each parametrization in train: ['0.08']
  - F1 Result for each parametrization in test: ['0.07']
  - precision Result for each parametrization in train: ['0.07']
  - precision for each parametrization in test: ['0.07']
  - recall Result for each parametrization in train: ['0.08']
  - recall Result for each parametrization in test: ['0.08']
Trying Tree ...
  - Best hyperparameter: {'Tree__max_depth': 7, 'Tree__random_state': 23}
  - Best F1 score: 0.721
  - F1 Result for each parametrization in train: ['0.82', '0.83', '0.83', '0.84', '0.85', '0.86', '0.87', '0.88', '0.89', '0.90', '0.92', '0.93', '0.94', '0.94', '0.95']
  - F1 Result for each parametrization in test: ['0.72', '0.69', '0.72', '0.71', '0.69', '0.67', '0.68', '0.65', '0.62', '0.64', '0.63', '0.62', '0.63', '0.62', '0.61']
  - precision Result for each parametrization in train: ['

In [48]:
results_japan[results_japan.algorithm == "Tree"].params.values

array([{'Tree__max_depth': 7, 'Tree__random_state': 23}], dtype=object)

In [50]:
pipeline = Pipeline([("scaler", StandardScaler()),("Tree", DecisionTreeClassifier())])
classifier = 

classifier = GridSearchCV(pipeline, param_grid=results_japan[results_japan.algorithm == "Tree"].params.values, cv=my_cv, scoring=['f1', 'precision', 'recall'], refit = 'f1', return_train_score = True)

ValueError: Parameter grid for parameter (Tree__max_depth) needs to be a list or numpy array, but got (<class 'int'>). Single values need to be wrapped in a list with one element.

In [ ]:
sklearn.model_selection.cross_val_predict(estimator, X, y=None, *, groups=None, cv=None, n_jobs=None, verbose=0, fit_params=None, pre_dispatch='2*n_jobs', method='predict')[source]¶

In [51]:
from sklearn.model_selection import cross_val_predict

In [ ]:
decision_tree = DecisionTreeClassifier()

In [66]:
len(df_japan_paper)

10185

In [57]:
results_japan[results_japan.algorithm == "Tree"].params.tolist()[0]

{'Tree__max_depth': 7, 'Tree__random_state': 23}

In [63]:
pipeline = Pipeline([("scaler", StandardScaler()),("Tree", DecisionTreeClassifier())])
pipeline = Pipeline([("Tree", DecisionTreeClassifier(max_depth = 7, random_state = 23))])
prediction_japan = cross_val_predict(pipeline, X, y = y, cv = my_cv)

In [65]:
len(X)

10185

In [64]:
prediction_japan.shape

(10185,)

In [94]:
np.isnan(prediction_japan).sum()

0

In [67]:
df_japan_paper["Collusive_competitor_prediction"] = prediction_japan

In [70]:
df_japan_paper_grouped.head()

Tender        Date  Site  Pre-Tender Estimate (PTE)  Competitors  \
0        2  1050537600   115                 14952000.0         1305   
10       3  1051228800   115                 23667000.0          125   
20       4  1052352000    84                 38800000.0         1293   
30       5  1052697600    56                 15800000.0          852   
50       7  1052870400   270                  8080000.0          794   

      Name competitors  Winner  Collusive_competitor  Bid_value  \
0       HEISEIKAIHATSU       0                     0        0.0   
10        GIMAKENSETSU       1                     0        0.0   
20       HAMASHINDOKEN       1                     0        0.0   
30  SANKOKENSETSUKOGYO       1                     0        0.0   
50    MINAMOTOKENSETSU       1                     0        0.0   

    Difference Bid/PTE  Number_bids Period investigation by JFTC      CV  \
0             0.784340           10               pre-inspection  0.1021   
10            0.801540           10               pre-inspection  0.0916   
20            0.985052           10               pre-inspection  0.0040   
30            0.970253           10               pre-inspection  0.0082   
50            0.968131           10               pre-inspection  0.0103   

       SPD   DIFFP      RD    KURT    SKEW  KSTEST  \
0   0.2982  0.0231  0.2375 -1.2043  0.8118  0.4030   
10  0.2515  0.0025  0.0245 -1.6380 -0.4335  0.2896   
20  0.0137  0.0065  1.6591  1.7464 -1.0795  0.3740   
30  0.0282  0.0027  0.4312  0.4611  0.6144  0.2453   
50  0.0314  0.0067  0.6470 -0.1258 -0.9291  0.3265   

    Collusive_competitor_original  
0                               0  
10                              0  
20                              0  
30                              0  
50                              0

In [71]:
df_japan_paper_grouped["Collusive_competitor_prediction"] = df_japan_paper.groupby(by="Tender").sum().reset_index().Collusive_competitor_prediction

/var/folders/kv/34jyv69d3lq7qnz8d_2l_s4w0000gq/T/ipykernel_9615/2326637853.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_japan_paper_grouped["Collusive_competitor_prediction"] = df_japan_paper.groupby(by="Tender").sum().reset_index().Collusive_competitor_prediction


Tender        Date  Site  Pre-Tender Estimate (PTE)  Competitors  \
0        2  1050537600   115                 14952000.0         1305   
10       3  1051228800   115                 23667000.0          125   
20       4  1052352000    84                 38800000.0         1293   
30       5  1052697600    56                 15800000.0          852   
50       7  1052870400   270                  8080000.0          794   

      Name competitors  Winner  Collusive_competitor  Bid_value  \
0       HEISEIKAIHATSU       0                     0        0.0   
10        GIMAKENSETSU       1                     0        0.0   
20       HAMASHINDOKEN       1                     0        0.0   
30  SANKOKENSETSUKOGYO       1                     0        0.0   
50    MINAMOTOKENSETSU       1                     0        0.0   

    Difference Bid/PTE  ...  Period investigation by JFTC      CV     SPD  \
0             0.784340  ...                pre-inspection  0.1021  0.2982   
10            0.801540  ...                pre-inspection  0.0916  0.2515   
20            0.985052  ...                pre-inspection  0.0040  0.0137   
30            0.970253  ...                pre-inspection  0.0082  0.0282   
50            0.968131  ...                pre-inspection  0.0103  0.0314   

     DIFFP      RD    KURT    SKEW  KSTEST  Collusive_competitor_original  \
0   0.0231  0.2375 -1.2043  0.8118  0.4030                              0   
10  0.0025  0.0245 -1.6380 -0.4335  0.2896                              0   
20  0.0065  1.6591  1.7464 -1.0795  0.3740                              0   
30  0.0027  0.4312  0.4611  0.6144  0.2453                              0   
50  0.0067  0.6470 -0.1258 -0.9291  0.3265                              0   

    Collusive_competitor_prediction  
0                               0.0  
10                              0.0  
20                              0.0  
30                              0.0  
50                              0.0  

[5 rows x 21 columns]

In [68]:
df_japan_paper_grouped["cartel_prediction"] = 

Tender        Date  Site  Pre-Tender Estimate (PTE)  Competitors  \
0       2  1050537600   115                 14952000.0         1305   
1       2  1050537600   115                 14952000.0          881   
2       2  1050537600   115                 14952000.0          685   
3       2  1050537600   115                 14952000.0          691   
4       2  1050537600   115                 14952000.0          125   

   Name competitors  Winner  Collusive_competitor  Bid_value  \
0    HEISEIKAIHATSU       0                     0   0.000000   
1     YAMAZATODOKEN       1                     0   0.204268   
2  MARUSARUKENSETSU       0                     0   0.226903   
3      MARUMORIGUMI       0                     0   0.251965   
4      GIMAKENSETSU       0                     0   0.255365   

   Difference Bid/PTE  ...  Period investigation by JFTC      CV     SPD  \
0            0.784340  ...                pre-inspection  0.1021  0.2982   
1            0.802484  ...                pre-inspection  0.1021  0.2982   
2            0.804494  ...                pre-inspection  0.1021  0.2982   
3            0.806721  ...                pre-inspection  0.1021  0.2982   
4            0.807022  ...                pre-inspection  0.1021  0.2982   

    DIFFP      RD    KURT    SKEW  KSTEST  Collusive_competitor_original  \
0  0.0231  0.2375 -1.2043  0.8118   0.403                              0   
1  0.0231  0.2375 -1.2043  0.8118   0.403                              0   
2  0.0231  0.2375 -1.2043  0.8118   0.403                              0   
3  0.0231  0.2375 -1.2043  0.8118   0.403                              0   
4  0.0231  0.2375 -1.2043  0.8118   0.403                              0   

   Collusive_competitor_prediction  
0                                0  
1                                0  
2                                0  
3                                0  
4                                0  

[5 rows x 21 columns]

In [86]:
df_japan_paper_grouped["n_competitors"] = df_japan_paper.groupby("Tender").head().Number_bids

/var/folders/kv/34jyv69d3lq7qnz8d_2l_s4w0000gq/T/ipykernel_9615/2613177475.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_japan_paper_grouped["n_competitors"] = df_japan_paper.groupby("Tender").head().Number_bids


In [107]:
df_japan_paper.groupby("Tender").sum().Collusive_competitor_prediction/df_japan_paper_grouped.n_competitors

0       NaN
2       NaN
3       NaN
4       NaN
5       NaN
         ..
13422   NaN
13452   NaN
13472   NaN
13483   NaN
13501   NaN
Length: 1564, dtype: float64

In [114]:
len(df_japan_paper_grouped.n_competitors)

810

In [118]:
cartel_prediction = [0 if perc < 0.11 else 1 for perc in (df_japan_paper.groupby("Tender").sum().Collusive_competitor_prediction.values)/(df_japan_paper_grouped.n_competitors)]

In [122]:
df_japan_paper_grouped["cartel_prediction"] = cartel_prediction

/var/folders/kv/34jyv69d3lq7qnz8d_2l_s4w0000gq/T/ipykernel_9615/2585504228.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_japan_paper_grouped["cartel_prediction"] = cartel_prediction


In [123]:
df_japan_paper_grouped.tail()

Tender        Date  Site  Pre-Tender Estimate (PTE)  Competitors  \
13422    1446  1174003200   139                262027500.0         1496   
13452    1449  1174003200   160                173943000.0          436   
13472    1450  1174003200    45                 16678200.0          316   
13483    1453  1174521600   203                 51439500.0          174   
13501    1454  1174521600    48                  3924900.0         1372   

       Name competitors  Winner  Collusive_competitor  Bid_value  \
13422  KYOEIKAIJIDOBOKU       1                     0        0.0   
13452        KOKUBAGUMI       0                     0        0.0   
13472   NAKAMURAKOMUTEN       1                     0        0.0   
13483         YUKIOGUMI       1                     0        0.0   
13501    YONAMINESANGYO       1                     0        0.0   

       Difference Bid/PTE  ...     SPD   DIFFP   RD     KURT    SKEW  KSTEST  \
13422            0.901623  ...  0.1093  0.0761  0.0   5.1260 -2.2325  0.5630   
13452            0.829995  ...  0.2000  0.0062  0.0   6.8519  2.8348  0.7790   
13472            0.950642  ...  0.0530  0.0000  0.0  -0.2748 -0.6773  0.2557   
13483            0.809757  ...  0.0870  0.0002  0.0   9.1625  2.7687  0.6546   
13501            0.968432  ...  0.0848  0.0083  0.0  12.7244  3.4937  0.7657   

       Collusive_competitor_original  Collusive_competitor_prediction  \
13422                              0                              NaN   
13452                              0                              NaN   
13472                              0                              NaN   
13483                              0                              NaN   
13501                              0                              NaN   

       n_competitors  cartel_prediction  
13422             12                  0  
13452             20                  0  
13472             11                  0  
13483             18                  0  
13501             14                  0  

[5 rows x 23 columns]

In [124]:
from sklearn.metrics import f1_score

In [125]:
f1_score(df_japan_paper_grouped.Collusive_competitor, df_japan_paper_grouped.cartel_prediction)

0.5693430656934306

In [126]:
print(len(df_japan_paper_grouped))

810


In [127]:
len(df_japan_paper)

10185

## Do CV by hand because something is strange

In [170]:
X.head()

Tender  Winner  Bid_value  Difference Bid/PTE  Number_bids      CV     SPD  \
0       2       0   0.000000            0.784340           10  0.1021  0.2982   
1       2       1   0.204268            0.802484           10  0.1021  0.2982   
2       2       0   0.226903            0.804494           10  0.1021  0.2982   
3       2       0   0.251965            0.806721           10  0.1021  0.2982   
4       2       0   0.255365            0.807022           10  0.1021  0.2982   

    DIFFP      RD    KURT    SKEW  KSTEST  
0  0.0231  0.2375 -1.2043  0.8118   0.403  
1  0.0231  0.2375 -1.2043  0.8118   0.403  
2  0.0231  0.2375 -1.2043  0.8118   0.403  
3  0.0231  0.2375 -1.2043  0.8118   0.403  
4  0.0231  0.2375 -1.2043  0.8118   0.403

In [172]:
print(len(y))
print(sum(y))

10185
805


In [176]:
len(df_japan_paper_grouped)

810

In [179]:
df_japan_paper_grouped.Collusive_competitor.sum()

94

In [181]:
y_grouped.sum()

94

# Ideas

Perhaps it would already help to add the number of colluded bidders per carteil into my ML algorithm. This would assign some sort of weight to the Statistical features. --> Test it out